<a href="https://colab.research.google.com/github/hannahestauss/Portfolio/blob/main/Disposable_Income.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction and Objective

Using databases on salaries by region and cost of living by region, I intend to paint a picture of the financial situations of different regions and the inequality of expendable income across those regions. Using salaries by region and cost of living by region, I will compare and contrast the value of each and how expensive they are in reality, adjusted based on regional changes in salary. 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import csv
from csv import reader
import re
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

Mounted at /content/gdrive


Loading the salaries CSV had some error because it required the encoding parameter be passed to the read_csv fundtion. -A

In [ ]:
cost_of_living = pd.read_csv("/content/gdrive/My Drive/StateCostOfLiving.csv",encoding= 'unicode_escape', sep=",", usecols=["State", "Liveable Wage"])

print(cost_of_living.head)

<bound method NDFrame.head of              State  Liveable Wage
0      Mississippi          46000
1         Arkansas          45000
2         Oklahoma          47000
3         Missouri          46000
4       New Mexico          48000
5        Tennessee          47000
6         Michigan          49000
7           Kansas          48000
8          Georgia          48000
9          Wyoming          48000
10         Alabama          46000
11         Indiana          47000
12            Iowa          49000
13            Ohio          46000
14        Nebraska          48000
15        Kentucky          43000
16   West Virginia          45000
17           Texas          48000
18           Idaho          46000
19       Louisiana          48000
20        Illinois          52000
21  North Carolina          50000
22  South Carolina          47000
23         Arizona          51000
24       Wisconsin          51000
25         Florida          52000
26            Utah          48000
27    North Dakota

In [ ]:
Wages = pd.read_csv("/content/gdrive/My Drive/salaries-by-region.csv",encoding= 'unicode_escape', sep=",", usecols=["Region", "Mid-Career Median Salary"])
Wages2= pd.read_csv("/content/gdrive/My Drive/salaries-by-region.csv",encoding= 'unicode_escape', sep=",", usecols=["Region", "Starting Median Salary"])
print(Wages2.head)

<bound method NDFrame.head of          Region Starting Median Salary
0    California            $70,400.00 
1    California            $75,500.00 
2    California            $71,800.00 
3    California            $59,900.00 
4    California            $51,900.00 
..          ...                    ...
315    New York            $38,000.00 
316    New York            $36,900.00 
317    New York            $37,800.00 
318       Maine            $39,400.00 
319    New York            $43,700.00 

[320 rows x 2 columns]>


# Data Cleaning and Preparation

In [ ]:
import re
Wages['Mid-Career Median Salary'] = Wages['Mid-Career Median Salary'].map(lambda x: x.lstrip('$').replace(",",""))
Wages2['Starting Median Salary'] = Wages2['Starting Median Salary'].map(lambda x: x.lstrip('$').replace(",",""))
#print(Wages)
Wages2.dropna(axis=0,inplace=True)
cost_of_living.dropna(axis=0,inplace=True)

*Requirement- regex*

In [ ]:
Wages2['Starting Median Salary'] = Wages2['Starting Median Salary'].astype(float)
cost_of_living['Liveable Wage'] = cost_of_living['Liveable Wage'].astype(float)

# Analysis Question #1: Which schools are worth it?

In [ ]:
with open('/content/gdrive/My Drive/salaries-by-region.csv', 'r',encoding= 'unicode_escape') as f:
    csv_reader = csv.DictReader(f)
    school_names = list(csv_reader)
school_names[3]  #,school_names[4]


OrderedDict([('School Name', 'University of California, Berkeley'),
             ('City', 'Berkeley'),
             ('Region', 'California'),
             ('Starting Median Salary', '$59,900.00 '),
             ('Mid-Career Median Salary', '$112,000.00 '),
             ('Mid-Career 10th Percentile Salary', '$59,500.00 '),
             ('Mid-Career 25th Percentile Salary', '$81,000.00 '),
             ('Mid-Career 75th Percentile Salary', '$149,000.00 '),
             ('Mid-Career 90th Percentile Salary', '$201,000.00 ')])

*Requirement: Lists*

In [ ]:
len(school_names)

320

In [ ]:
State = "California"
total_Salary = 0
Number_of_schools = 0
for school in school_names:
     #print(f"{school['School Name']} \t {school['Region']}: \t\t  {school['Starting Median Salary']}")
    if school['Region'] == State:
        #print("true")
        Number_of_schools += 1
        slary = school['Starting Median Salary']
        slary = slary.replace(",","")
        slary = slary.replace("$","")
        total_Salary +=float(slary)

State_Avg = total_Salary/Number_of_schools

print("The total salary for this state is : ", "$", total_Salary)
print("The average starting salary in this state is: ", "$", State_Avg)

#Use the school's index to test the worthiness of the school, in this example the [2] index is harvey mudd college
school = school_names[2]
if float(school['Starting Median Salary'].replace("$","").replace(",","")) > State_Avg:
    print(f"{school['School Name']} average starting salary is more than the state's starting salary, this school is worth it")
else:
    print(f"{school['School Name']} average starting salary is less than the state's starting salary, this school isn't worth it")
print(f"Starting Median Salary for this school:  {school['Starting Median Salary']}")

The total salary for this state is :  $ 1428900.0
The average starting salary in this state is:  $ 51032.142857142855
Harvey Mudd College average starting salary is more than the state's starting salary, this school is worth it
Starting Median Salary for this school:  $71,800.00 


In [ ]:
Avg_Salaries = dict()
states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
for x in states:
  Wage = Wages.loc[Wages['Region'].isin([(x)])]
  Avg_Salaries[x] = Wage.mean()



# Analysis Question #2: Which states have the best salary/living cost balance?

*Requirements: Iteration, Dictionaries, Strings, Data Visualization*

In [ ]:
disp_income = dict()
states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
for x in states:
  Wage = Wages2.loc[Wages2['Region'].isin([(x)])]
  #print("The average starting median salary is {}".format(Wage.mean().values))
  Rent = cost_of_living.loc[cost_of_living['State'].isin([(x)])]
  #print("The median rent is {}".format(Rent.mean().values))
  disposable_income = ((Wage.mean().values)-(Rent.mean().values))
  #print("The disposable income for " + x + " is ")
  #print(disposable_income)
  disp_income[x] = disposable_income
  #salarybystate[x] = (Wage.mean().values)
print (disp_income)
#print (salarybystate)

{'Alabama': array([-3750.]), 'Alaska': array([-8100.]), 'Arizona': array([-3550.]), 'Arkansas': array([-4333.33333333]), 'California': array([-5967.85714286]), 'Colorado': array([-7585.71428571]), 'Connecticut': array([-10714.28571429]), 'Delaware': array([-7100.]), 'Florida': array([-9455.55555556]), 'Georgia': array([1050.]), 'Hawaii': array([-17200.]), 'Idaho': array([-2466.66666667]), 'Illinois': array([-5040.]), 'Indiana': array([-1283.33333333]), 'Iowa': array([-4766.66666667]), 'Kansas': array([-5966.66666667]), 'Kentucky': array([-4200.]), 'Louisiana': array([-2300.]), 'Maine': array([-6520.]), 'Maryland': array([-8500.]), 'Massachusetts': array([-9857.89473684]), 'Michigan': array([-4983.33333333]), 'Minnesota': array([-7233.33333333]), 'Mississippi': array([-5566.66666667]), 'Missouri': array([-2750.]), 'Montana': array([-6400.]), 'Nebraska': array([-4400.]), 'Nevada': array([-7150.]), 'New Hampshire': array([-13200.]), 'New Jersey': array([-4900.]), 'New Mexico': array([-236

In [ ]:
DispData2 = pd.DataFrame(list(disp_income.items()),columns = ['State','Disposable Income'])
DispData2['Disposable Income'] = DispData2['Disposable Income'].astype(float)

In [ ]:
DispData = pd.DataFrame(list(disp_income.items()),columns = ['State','Disposable Income'])
DispData['Disposable Income'] = DispData['Disposable Income'].astype(float)
print (DispData)

             State  Disposable Income
0          Alabama       -3750.000000
1           Alaska       -8100.000000
2          Arizona       -3550.000000
3         Arkansas       -4333.333333
4       California       -5967.857143
5         Colorado       -7585.714286
6      Connecticut      -10714.285714
7         Delaware       -7100.000000
8          Florida       -9455.555556
9          Georgia        1050.000000
10          Hawaii      -17200.000000
11           Idaho       -2466.666667
12        Illinois       -5040.000000
13         Indiana       -1283.333333
14            Iowa       -4766.666667
15          Kansas       -5966.666667
16        Kentucky       -4200.000000
17       Louisiana       -2300.000000
18           Maine       -6520.000000
19        Maryland       -8500.000000
20   Massachusetts       -9857.894737
21        Michigan       -4983.333333
22       Minnesota       -7233.333333
23     Mississippi       -5566.666667
24        Missouri       -2750.000000
25         M

In [ ]:
df1 = DispData[DispData["Disposable Income"] < 30111.976]
print("These states fall below the average disposable income\n")
print(df1)

These states fall below the average disposable income

             State  Disposable Income
0          Alabama       -3750.000000
1           Alaska       -8100.000000
2          Arizona       -3550.000000
3         Arkansas       -4333.333333
4       California       -5967.857143
5         Colorado       -7585.714286
6      Connecticut      -10714.285714
7         Delaware       -7100.000000
8          Florida       -9455.555556
9          Georgia        1050.000000
10          Hawaii      -17200.000000
11           Idaho       -2466.666667
12        Illinois       -5040.000000
13         Indiana       -1283.333333
14            Iowa       -4766.666667
15          Kansas       -5966.666667
16        Kentucky       -4200.000000
17       Louisiana       -2300.000000
18           Maine       -6520.000000
19        Maryland       -8500.000000
20   Massachusetts       -9857.894737
21        Michigan       -4983.333333
22       Minnesota       -7233.333333
23     Mississippi       -5566.66

In [ ]:
column = DispData["Disposable Income"]
max_value = column.max()
df2 = DispData[DispData["Disposable Income"] == max_value]
print("The best-valued place to live is: ")
print(df2)

The best-valued place to live is: 
     State  Disposable Income
9  Georgia             1050.0


In [ ]:
column = DispData["Disposable Income"]
min_value = column.min()
df3 = DispData[DispData["Disposable Income"] == min_value]
print("The worst-valued place to live is: ")
print(df3)

The worst-valued place to live is: 
     State  Disposable Income
10  Hawaii           -17200.0


# Analysis Question #3: Which states satisfy economists' advice?

*Requirements: Dictionaries, Iteration, String, Functions, Databases*

In [ ]:
costofliving = pd.read_csv("/content/gdrive/My Drive/StateCostOfLiving.csv",encoding= 'unicode_escape', sep=",", usecols=["State", "Median Rent"])
costofliving['Median Rent'] = costofliving['Median Rent'].astype(float)
Avg_Salaries = dict()
states = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
for x in states:
  Wage2 = Wages2.loc[Wages2['Region'].isin([(x)])]
  Avg_Salaries[x] = Wage2.mean()


In [ ]:
Percent = pd.DataFrame(list(Avg_Salaries.items()),columns = ['State','Average Salary'])
Percent['Average Salary'] = Percent['Average Salary'].astype(float)
Percent['Rent Allocation'] = Percent['Average Salary']*0.3
Percent['Actual Rent'] = costofliving['Median Rent']*12
Percent['Percentage'] = (Percent['Actual Rent']/Percent['Average Salary'])
print (Percent)

             State  Average Salary  Rent Allocation  Actual Rent  Percentage
0          Alabama    42250.000000     12675.000000       9360.0    0.221538
1           Alaska    45900.000000     13770.000000       8940.0    0.194771
2          Arizona    47450.000000     14235.000000       9720.0    0.204847
3         Arkansas    40666.666667     12200.000000       9960.0    0.244918
4       California    51032.142857     15309.642857      10128.0    0.198463
5         Colorado    46414.285714     13924.285714      10428.0    0.224672
6      Connecticut    49285.714286     14785.714286      10452.0    0.212070
7         Delaware    45900.000000     13770.000000      10200.0    0.222222
8          Florida    42544.444444     12763.333333      12072.0    0.283750
9          Georgia    49050.000000     14715.000000      10260.0    0.209174
10          Hawaii    43800.000000     13140.000000       9504.0    0.216986
11           Idaho    43533.333333     13060.000000       9912.0    0.227688

In [ ]:
d1 = Percent[Percent["Percentage"] > 0.3]
print(d1)

             State  Average Salary  Rent Allocation  Actual Rent  Percentage
23     Mississippi    40433.333333     12130.000000      12624.0    0.312218
25         Montana    40600.000000     12180.000000      14100.0    0.347291
32  North Carolina    43688.888889     13106.666667      15252.0    0.349105
34            Ohio    43100.000000     12930.000000      13560.0    0.314617
35        Oklahoma    42475.000000     12742.500000      13284.0    0.312749
36          Oregon    41860.000000     12558.000000      13332.0    0.318490
37    Pennsylvania    50255.555556     15076.666667      15096.0    0.300385
41       Tennessee    41671.428571     12501.428571      16008.0    0.384148
42           Texas    46946.153846     14083.846154      14160.0    0.301622
43            Utah    44580.000000     13374.000000      16704.0    0.374697
44         Vermont    46250.000000     13875.000000      14928.0    0.322768
45        Virginia    46925.000000     14077.500000      15384.0    0.327842